In [ ]:
#todo Añadir deflactor del PIB trimestral


In [2]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict # Útil para recolectar datos a nivel estatal

# ----------------------------------------------------------------------
# FUNCIONES DE UTILIDAD
# ----------------------------------------------------------------------

def weighted_average(df, value_col, weight_col):
    """Calcula el promedio ponderado de una columna usando pesos (factores de expansión)."""
    # Asegura que las columnas de valor y peso existan y no sean NaN
    df_filtered = df.dropna(subset=[value_col, weight_col])
    
    # Maneja el caso de que no haya datos o la suma de pesos sea cero
    if df_filtered.empty or df_filtered[weight_col].sum() == 0:
        return np.nan
    
    # Excluir valores negativos o cero si se asume que 'ingocup' es ingreso positivo
    # if value_col in ['ingocup', 'ing_x_hrs']:
    #     df_filtered = df_filtered[df_filtered[value_col] > 0]
    
    return np.average(df_filtered[value_col], weights=df_filtered[weight_col])

# Diccionario de Entidades para mapear códigos a nombres
ENTIDADES = {
    1: 'Aguascalientes', 2: 'Baja California', 3: 'Baja California Sur', 4: 'Campeche',
    5: 'Coahuila', 6: 'Colima', 7: 'Chiapas', 8: 'Chihuahua', 9: 'Ciudad de México',
    10: 'Durango', 11: 'Guanajuato', 12: 'Guerrero', 13: 'Hidalgo', 14: 'Jalisco',
    15: 'México', 16: 'Michoacán', 17: 'Morelos', 18: 'Nayarit', 19: 'Nuevo León',
    20: 'Oaxaca', 21: 'Puebla', 22: 'Querétaro', 23: 'Quintana Roo', 24: 'San Luis Potosí',
    25: 'Sinaloa', 26: 'Sonora', 27: 'Tabasco', 28: 'Tamaulipas', 29: 'Tlaxcala',
    30: 'Veracruz', 31: 'Yucatán', 32: 'Zacatecas'
}

# ----------------------------------------------------------------------
# FUNCIÓN PRINCIPAL DE PROCESAMIENTO TRIMESTRAL
# ----------------------------------------------------------------------

def procesar_trimestre_enoe(year, quarter, file_format='dta'):
    """
    Carga, limpia y calcula indicadores clave a nivel nacional y estatal 
    para un trimestre específico.
    
    Args:
        year (int): El año del trimestre a analizar (e.g., 2023).
        quarter (int): El número de trimestre (1, 2, 3, 4).
        file_format (str): Formato del archivo ('dta' o 'csv').
        
    Returns:
        tuple: (pd.Series Nacional, pd.DataFrame Estatal) con los indicadores, 
               o (None, None) si el archivo no se encuentra o está vacío.
    """
    periodo_str = f"{year} T{quarter}"
    print(f"\n--- ⏳ Procesando: {periodo_str} ---")

    # --- 1. Construcción de la Ruta del Archivo ---
    year_short = str(year)[-2:]
    dir_name = f"ENOE_{year}_{quarter}"
    file_name = f"ENOE_SDEMT{quarter}{year_short}.{file_format}"
    file_path = os.path.join("data", f"ENOE_{file_format}", dir_name, file_name)
    
    # --- 2. Carga de Datos y Manejo de Errores (Debugging) ---
    if not os.path.exists(file_path):
        print(f"❌ Error Crítico: Archivo no encontrado en: {file_path}")
        return None, None # Retorna None para el control en el script principal
    
    try:
        if file_format == 'dta':
            df = pd.read_stata(file_path, convert_categoricals=False)
        elif file_format == 'csv':
            df = pd.read_csv(file_path)
        else:
            raise ValueError("Formato de archivo no soportado.")
        
        if df.empty:
            print(f"❌ Error de Carga: Archivo encontrado, pero vacío: {file_path}")
            return None, None
            
        print(f"✅ Archivo cargado exitosamente. {len(df):,} registros.")
        
    except Exception as e:
        print(f"❌ Ocurrió un error de lectura de datos: {e}")
        return None, None

    # --- 3. Limpieza y Preparación de Datos ---
    # Conversión de tipos de datos esenciales
    df['r_def'] = df['r_def'].astype(str).str.strip()
    
    columnas_numericas = [
        'fac_tri', 'sex', 'eda', 'clase1', 'clase2', 'c_res',
        'ingocup', 'ing_x_hrs', 'ent'
    ]
    for col in columnas_numericas:
        # Usamos errors='coerce' para convertir valores no numéricos a NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Filtro base: Universo de residentes con entrevista completa (r_def='00', c_res=1 o 3)
    df_base = df[(df['r_def'] == '0.0') & (df['c_res'].isin([1, 3]))].copy()
    
    if df_base.empty:
        print("❌ Error de Filtro: No se encontraron registros válidos después del filtro base.")
        return None, None

    # Filtro de Población en Edad de Trabajar (PET): 15 años y más
    df_15_y_mas = df_base[df_base['eda'].between(15, 98)].copy()
    
    # Definición de subconjuntos
    df_pea = df_15_y_mas[df_15_y_mas['clase1'] == 1].copy()      # PEA (clase1=1)
    df_ocupada = df_15_y_mas[df_15_y_mas['clase2'] == 1].copy()  # Ocupada (clase2=1)
    
    # Asignación de nombres de estado (Necesario para ambos niveles)
    df_base['ent_nombre'] = df_base['ent'].map(ENTIDADES)
    df_15_y_mas['ent_nombre'] = df_15_y_mas['ent'].map(ENTIDADES)
    df_pea['ent_nombre'] = df_pea['ent'].map(ENTIDADES)
    df_ocupada['ent_nombre'] = df_ocupada['ent'].map(ENTIDADES)

    # ------------------------------------------------------------------
    # --- 4. CÁLCULOS A NIVEL NACIONAL ---
    # ------------------------------------------------------------------
    
    datos_nacional = {
        # Identificadores de Tiempo
        'year': year,
        'quarter': quarter,
        
        # Población Total
        'pob_total': df_base['fac_tri'].sum(),
        'pob_hombres_total': df_base[df_base['sex'] == 1]['fac_tri'].sum(),
        'pob_mujeres_total': df_base[df_base['sex'] == 2]['fac_tri'].sum(),
        
        # PET (15 años y más)
        'pet_total': df_15_y_mas['fac_tri'].sum(),
        'pet_hombres_15mas': df_15_y_mas[df_15_y_mas['sex'] == 1]['fac_tri'].sum(),
        'pet_mujeres_15mas': df_15_y_mas[df_15_y_mas['sex'] == 2]['fac_tri'].sum(),

        # PEA
        'pea_total': df_pea['fac_tri'].sum(),
        'pea_hombres': df_pea[df_pea['sex'] == 1]['fac_tri'].sum(),
        'pea_mujeres': df_pea[df_pea['sex'] == 2]['fac_tri'].sum(),
        
        # Ingreso Promedio
        'ing_prom_mes_total': weighted_average(df_ocupada, 'ingocup', 'fac_tri'),
        'ing_prom_mes_hombres': weighted_average(df_ocupada[df_ocupada['sex'] == 1], 'ingocup', 'fac_tri'),
        'ing_prom_mes_mujeres': weighted_average(df_ocupada[df_ocupada['sex'] == 2], 'ingocup', 'fac_tri'),
        
        'ing_prom_hora_total': weighted_average(df_ocupada, 'ing_x_hrs', 'fac_tri'),
        'ing_prom_hora_hombres': weighted_average(df_ocupada[df_ocupada['sex'] == 1], 'ing_x_hrs', 'fac_tri'),
        'ing_prom_hora_mujeres': weighted_average(df_ocupada[df_ocupada['sex'] == 2], 'ing_x_hrs', 'fac_tri'),
    }
    
    # ------------------------------------------------------------------
    # --- 5. CÁLCULOS A NIVEL ESTATAL ---
    # ------------------------------------------------------------------
    
    # Inicialización de un diccionario de listas para recolectar datos por estado
    datos_estatal = defaultdict(list)
    
    for ent_code, ent_name in ENTIDADES.items():
        # Filtros por Estado
        df_base_est = df_base[df_base['ent'] == ent_code]
        df_15_y_mas_est = df_15_y_mas[df_15_y_mas['ent'] == ent_code]
        df_pea_est = df_pea[df_pea['ent'] == ent_code]
        df_ocupada_est = df_ocupada[df_ocupada['ent'] == ent_code]
        
        # Recolección de datos
        datos_estatal['year'].append(year)
        datos_estatal['quarter'].append(quarter)
        datos_estatal['ent_code'].append(ent_code)
        datos_estatal['ent_nombre'].append(ent_name)
        
        # Población Total
        datos_estatal['pob_total'].append(df_base_est['fac_tri'].sum())
        datos_estatal['pob_hombres_total'].append(df_base_est[df_base_est['sex'] == 1]['fac_tri'].sum())
        datos_estatal['pob_mujeres_total'].append(df_base_est[df_base_est['sex'] == 2]['fac_tri'].sum())

        # PET (15 años y más)
        datos_estatal['pet_hombres_15mas'].append(df_15_y_mas_est[df_15_y_mas_est['sex'] == 1]['fac_tri'].sum())
        datos_estatal['pet_mujeres_15mas'].append(df_15_y_mas_est[df_15_y_mas_est['sex'] == 2]['fac_tri'].sum())

        # PEA
        datos_estatal['pea_hombres'].append(df_pea_est[df_pea_est['sex'] == 1]['fac_tri'].sum())
        datos_estatal['pea_mujeres'].append(df_pea_est[df_pea_est['sex'] == 2]['fac_tri'].sum())
        
        # Ingreso Promedio
        datos_estatal['ing_prom_mes_total'].append(weighted_average(df_ocupada_est, 'ingocup', 'fac_tri'))
        datos_estatal['ing_prom_hora_total'].append(weighted_average(df_ocupada_est, 'ing_x_hrs', 'fac_tri'))

    # Convierte el diccionario recolectado a un DataFrame
    df_estatal_trimestre = pd.DataFrame(datos_estatal)
    
    return pd.Series(datos_nacional), df_estatal_trimestre

# ----------------------------------------------------------------------
# EJECUCIÓN DEL SCRIPT Y CONSOLIDACIÓN DE SERIES DE TIEMPO
# ----------------------------------------------------------------------

if __name__ == "__main__":
    # --- RANGO DE ANÁLISIS ---
    # Define el rango de años y trimestres a analizar
    START_YEAR = 2018
    END_YEAR = 2024
    
    # Lista para almacenar los resultados nacionales (Series de Pandas)
    resultados_nacionales = []
    # Lista para almacenar los resultados estatales (DataFrames)
    resultados_estatales = []

    # Genera la secuencia de trimestres
    periodos = []
    for y in range(START_YEAR, END_YEAR + 1):
        for q in range(1, 5):
            # Condición para saltarse trimestres no disponibles (ej. 2020 T2 y T3)
            if y == 2020 and q in [2, 3]:
                print(f"--- ⚠️ Saltando periodo {y} T{q} (No disponible o no oficial). ---")
                continue
            periodos.append((y, q))

    print(f"\n===========================================================")
    print(f"  INICIANDO PROCESAMIENTO DE {len(periodos)} TRIMESTRES")
    print(f"  Rango: {START_YEAR} T1 hasta {END_YEAR} T4")
    print(f"===========================================================")

    # Bucle principal para procesar cada trimestre
    for year, quarter in periodos:
        df_nacional, df_estatal = procesar_trimestre_enoe(year, quarter)
        
        if df_nacional is not None and df_estatal is not None:
            resultados_nacionales.append(df_nacional)
            resultados_estatales.append(df_estatal)
        else:
            # Manejo explícito de trimestres sin datos (se añade una fila con NA)
            periodo_na = {'year': year, 'quarter': quarter}
            
            # Series Nacional con NA
            serie_na_nacional = pd.Series(periodo_na)
            resultados_nacionales.append(serie_na_nacional)
            
            # DataFrame Estatal con NA
            df_na_estatal = pd.DataFrame(periodo_na, index=range(1, 33)) # 32 estados
            df_na_estatal['ent_code'] = df_na_estatal.index
            df_na_estatal['ent_nombre'] = df_na_estatal['ent_code'].map(ENTIDADES)
            # Rellenar todas las columnas de variables con NaN
            for col in resultados_estatales[0].columns if resultados_estatales else []:
                if col not in df_na_estatal.columns:
                    df_na_estatal[col] = np.nan
            resultados_estatales.append(df_na_estatal)
            
            print(f"--- 🚫 Se agregó NA/NaN para {year} T{quarter} y se prosigue. ---")
            

    # --- 6. CONSOLIDACIÓN DE BASES DE DATOS ---

    # 1. Serie de Tiempo Nacional
    df_serie_nacional = pd.DataFrame(resultados_nacionales).reset_index(drop=True)
    # Crea un índice de tiempo para facilitar el análisis
    df_serie_nacional['periodo'] = df_serie_nacional['year'].astype(str) + '-T' + df_serie_nacional['quarter'].astype(str)
    df_serie_nacional.set_index('periodo', inplace=True)

    print("\n===========================================================")
    print("      ✅ BASE DE SERIE DE TIEMPO NACIONAL CREADA")
    print("===========================================================")
    print(df_serie_nacional.head())
    df_serie_nacional.to_csv("Resultados/serie_tiempo_nacional.csv")


    # 2. Serie de Tiempo Estatal
    df_serie_estatal = pd.concat(resultados_estatales, ignore_index=True)
    # Crea un índice de tiempo
    df_serie_estatal['periodo'] = df_serie_estatal['year'].astype(str) + '-T' + df_serie_estatal['quarter'].astype(str)
    
    print("\n===========================================================")
    print("      ✅ BASE DE SERIE DE TIEMPO ESTATAL CREADA")
    print("===========================================================")
    print(df_serie_estatal.head())
    df_serie_estatal.to_csv("Resultados/serie_tiempo_estatal.csv")

--- ⚠️ Saltando periodo 2020 T2 (No disponible o no oficial). ---
--- ⚠️ Saltando periodo 2020 T3 (No disponible o no oficial). ---

  INICIANDO PROCESAMIENTO DE 26 TRIMESTRES
  Rango: 2018 T1 hasta 2024 T4

--- ⏳ Procesando: 2018 T1 ---
❌ Error Crítico: Archivo no encontrado en: data\ENOE_dta\ENOE_2018_1\ENOE_SDEMT118.dta
--- 🚫 Se agregó NA/NaN para 2018 T1 y se prosigue. ---

--- ⏳ Procesando: 2018 T2 ---
❌ Error Crítico: Archivo no encontrado en: data\ENOE_dta\ENOE_2018_2\ENOE_SDEMT218.dta
--- 🚫 Se agregó NA/NaN para 2018 T2 y se prosigue. ---

--- ⏳ Procesando: 2018 T3 ---
❌ Error Crítico: Archivo no encontrado en: data\ENOE_dta\ENOE_2018_3\ENOE_SDEMT318.dta
--- 🚫 Se agregó NA/NaN para 2018 T3 y se prosigue. ---

--- ⏳ Procesando: 2018 T4 ---
❌ Error Crítico: Archivo no encontrado en: data\ENOE_dta\ENOE_2018_4\ENOE_SDEMT418.dta
--- 🚫 Se agregó NA/NaN para 2018 T4 y se prosigue. ---

--- ⏳ Procesando: 2019 T1 ---
❌ Error Crítico: Archivo no encontrado en: data\ENOE_dta\ENOE_2019_1\E